score reaction and subsystem pathways on Th17 Compass data  
different filtering for min number of genes in pathway

In [1]:
import pandas as pd
import scanpy as sc
import sys
import numpy as np
import scipy
import joblib
import matplotlib.pyplot as plt
from utils import *

In [2]:
out_file_path = './'
data_dir = out_file_path + 'data/'
gene_signature_dir = out_file_path + 'gene_signatures/'
out_dir = out_file_path + 'compass_hypergeom_results/'

From DESeq2, look for enrichment of metabolism terms in DE genes

**read gene signatures**

In [3]:
out_filename = gene_signature_dir + 'metabolism_rxn_subsystem.pickle'
subsystem_to_genes = joblib.load(out_filename)
metabolism_sig = {key: [item[0] for item in subsystem_to_genes[key]] for key in subsystem_to_genes}

In [7]:
out_filename = gene_signature_dir + 'metabolism_rxn_reaction.pickle'
reaction_sig_genes = joblib.load(out_filename)
reaction_sig = {k: v[0] for k,v in reaction_sig_genes.items()}

In [9]:
for k,v in metabolism_sig.items():
    print(f'{k}: {len(v)}')

Fatty acid oxidation: 32
Nucleotide interconversion: 102
Keratan sulfate degradation: 11
Sphingolipid metabolism: 29
Tyrosine metabolism: 34
Keratan sulfate synthesis: 15
Steroid metabolism: 25
Chondroitin synthesis: 30
Tryptophan metabolism: 36
Glycerophospholipid metabolism: 61
Inositol phosphate metabolism: 60
Folate metabolism: 11
Valine, leucine, and isoleucine metabolism: 31
Chondroitin sulfate degradation: 10
Cholesterol metabolism: 22
Arginine and Proline Metabolism: 33
Bile acid synthesis: 9
Fatty acid synthesis: 8
Glycine, serine, alanine and threonine metabolism: 23
Pyrimidine catabolism: 23
Blood group synthesis: 5
Pyruvate metabolism: 32
Aminosugar metabolism: 26
Heparan sulfate degradation: 6
Urea cycle: 17
Purine catabolism: 13
Eicosanoid metabolism: 19
Methionine and cysteine metabolism: 20
Lysine metabolism: 25
Starch and sucrose metabolism: 14
Pentose phosphate pathway: 12
NAD metabolism: 14
Vitamin A metabolism: 15
beta-Alanine metabolism: 21
Triacylglycerol synthesi

In [11]:
print(len(metabolism_sig.keys()))

81


In [12]:
for k,v in reaction_sig.items():
    print(f'{k}: {len(v)}')

13DAMPPOX: 3
2HBO: 5
2OXOADOXm: 4
34DHOXPEGOX: 6
34DHPHAMT: 1
34DHPLACOX: 4
34DHPLACOX_NADP_: 3
34DHXMANDACOX: 3
34DHXMANDACOX_NADP_: 2
34HPPOR: 1
3AIBTm: 1
3DSPHR: 1
3HAO: 1
3HBCDm: 1
3HKYNAKGAT: 2
3HLYTCL: 1
3HPCOAHYD: 1
3HXKYNOXDA: 2
3M4HDXPAC: 4
3MOX4HOXPGALDOX: 4
3MOX4HOXPGALDOX_NADP_: 4
3MOXTYROX: 2
3NTD7l: 4
3SALACBOXL: 3
3SALATAi: 1
3SALATAim: 1
41R1H2MAE12BOOX: 2
41R2A1H12BOOX: 2
42A12BOOX: 5
4HGLSDm: 1
4HOXPACDOX_NADP_: 4
4NPHSULT: 1
5ADTSTSTERONESULT: 2
5HLTDL: 1
5HOXINDACTO2OX: 1
5HOXINDACTOXm: 3
5HOXINOXDA: 2
5HTRPDOX: 1
5HXKYNDCL: 1
5HXKYNOXDA: 2
A4GALTc: 1
A4GALTg: 1
AACOAT: 1
AACTOOR: 1
AATAi: 1
ABTArm: 1
ABUTD: 1
ACACT10m: 3
ACACT1x: 1
ACACT4p: 1
ACACT5p: 1
ACACT6p: 1
ACACT7p: 1
ACACT8p: 1
ACACT9p: 1
ACCOACm: 1
ACCOAL: 1
ACCOALm: 2
ACGAM2E: 1
ACGAM6PSi: 1
ACGAMK: 1
ACGAMPM: 1
ACGPID: 1
ACGSm: 1
ACHEe: 1
ACNAM9PL: 1
ACNAM9PL2: 1
ACNAMPH: 3
ACOAD10m: 2
ACOAD1fm: 7
ACOAD8m: 1
ACOAD9m: 2
ACOAO7p: 1
ACOATA: 1
ACODA: 1
ACOX22x: 1
ACOX2x: 1
ACP1_FMN_: 4
ACS: 2
ACS2: 2
ACSm: 1

In [13]:
print(len(reaction_sig.keys()))

1292


In [14]:
# read in DE genes
compass_data = pd.read_csv(data_dir + 'th_data/GSE162300_DFMO_RNA_est_counts.csv', sep = ',', index_col = 0) # this is library size normalized already
samples = list(set([i[:-9] for i in compass_data.columns]))

In [15]:
n_genes_total = len(compass_data.index)
n_genes_total

20817

In [16]:
# these are for thP vs N

In [17]:
de_genes_th_1 = pd.read_csv(data_dir + 'th_data/deseq_genes_th_uncollapse.csv', index_col = 0)
#de_genes_th_2 = pd.read_csv(data_dir + 'th_data/deseq_genes_th_collapse.csv', index_col = 0)

In [19]:
logfc_thresh = 0
p_thresh = 0.05

# for DE genes of Th17p vs Th17n

# subsystem scoring

In [20]:
all_genes = list(compass_data.index)
overlapping_terms, n_removed_terms = get_overlapping_met_genes(metabolism_sig, all_genes, verb = True)

Fatty acid oxidation	original: 32 | overlapping: 32 | removed: 0
Nucleotide interconversion	original: 102 | overlapping: 101 | removed: 1
Keratan sulfate degradation	original: 11 | overlapping: 11 | removed: 0
Sphingolipid metabolism	original: 29 | overlapping: 29 | removed: 0
Tyrosine metabolism	original: 34 | overlapping: 33 | removed: 1
Keratan sulfate synthesis	original: 15 | overlapping: 14 | removed: 1
Steroid metabolism	original: 25 | overlapping: 24 | removed: 1
Chondroitin synthesis	original: 30 | overlapping: 30 | removed: 0
Tryptophan metabolism	original: 36 | overlapping: 35 | removed: 1
Glycerophospholipid metabolism	original: 61 | overlapping: 59 | removed: 2
Inositol phosphate metabolism	original: 60 | overlapping: 60 | removed: 0
Folate metabolism	original: 11 | overlapping: 10 | removed: 1
Valine, leucine, and isoleucine metabolism	original: 31 | overlapping: 31 | removed: 0
Chondroitin sulfate degradation	original: 10 | overlapping: 10 | removed: 0
Cholesterol metabol

filter to at least 5 genes in signature & 70% of genes are retained

In [27]:
all_genes = list(compass_data.index)
overlapping_terms, n_removed_terms = get_overlapping_met_genes(metabolism_sig, all_genes, verb = False)

In [21]:
prop_removed = {}
for k,v in n_removed_terms.items():
    prop_removed[k] = (len(metabolism_sig[k]) - v)/len(metabolism_sig[k])
#plt.hist(prop_removed.values())

In [29]:
min_prop = 0.7
upper_range = 16
overlapping_terms_filter_all = {i:{} for i in range(1,upper_range)}
for min_length in range(1,upper_range):
    overlapping_terms_filter = {}
    for k,v in overlapping_terms.items():
        if (len(v) >= min_length) & (prop_removed[k] > min_prop):
            overlapping_terms_filter[k] = v
    overlapping_terms_filter_all[min_length] = overlapping_terms_filter

In [30]:
for k,v in overlapping_terms_filter_all.items():
    print(f'{k}: {len(v)}')

1: 81
2: 74
3: 68
4: 66
5: 60
6: 58
7: 54
8: 51
9: 50
10: 48
11: 45
12: 41
13: 37
14: 31
15: 29


In [41]:
def get_hypergeometric_results_compass_data(de_genes, metabolism, M = n_genes_total, p_thresh = p_thresh, logfc_thresh = logfc_thresh):
    res = {k: {} for k in ['pvn', 'nvp']}
    for key in ['pvn', 'nvp']:
        df = de_genes.copy()
        df = df[df['padj'] < p_thresh]
        if key == 'pvn':
            df = df[df['log2FoldChange'] > logfc_thresh]
        else:
            df = df[df['log2FoldChange'] < logfc_thresh]
            
        df['genes'] = df.index.str.upper()
        df_genes = set(df['genes'])
        N = len(df) # length of DE genes
        for k,v in metabolism.items():
            n = len(v) # number of met genes
            x = len(df_genes.intersection(v))
            pct_overlap = x/(N+n)

            if pct_overlap > 0:
                p_val = hypergeometric_test(total_genes_expressed=M, n_genes_of_interest=n, 
                                            n_genes_picked=N, n_overlap=x)
                res[key][k] = p_val
#             else:
#                 res[key][k] = 1
    return res

In [45]:
def format_hypergeom_res(res):
    res_df = pd.DataFrame.from_dict(res)
    res_df['term'] = res_df.index
    res_df['padj_pvn'] = adjust_p_value_bh(res_df['pvn'].values)
    res_df['padj_nvp'] = adjust_p_value_bh(res_df['nvp'].values)
    res_df['q_pvn'] = adjust_p_value_qval(res_df['pvn'].values,pi_0=None)
    res_df['q_nvp'] = adjust_p_value_qval(res_df['nvp'].values,pi_0=None)
    
    cols_pvn = ['pvn', 'term', 'padj_pvn', 'q_pvn']
    res_df_format = res_df[cols_pvn].reset_index(drop = True).copy()
    res_df_format.columns = ['p', 'term', 'padj', 'q']
    res_df_format = res_df_format[['term', 'p', 'padj', 'q']]
    res_pvn = res_df_format.copy()
    
    cols_nvp = ['nvp', 'term', 'padj_nvp', 'q_nvp']
    res_df_format = res_df[cols_nvp].reset_index(drop = True).copy()
    res_df_format.columns = ['p', 'term', 'padj', 'q']
    res_df_format = res_df_format[['term', 'p', 'padj', 'q']]
    res_nvp = res_df_format.copy()
    
    return res_pvn, res_nvp

In [67]:
res_pvn_all = {}
res_nvp_all = {}
for k,v in overlapping_terms_filter_all.items():
    res_pvn_all[k], res_nvp_all[k] = format_hypergeom_res(get_hypergeometric_results_compass_data(de_genes_th_1, v))

In [106]:
res_pvn_all[1][res_pvn_all[1]['q'] < 0.05]

,term,p,padj,q
1,Nucleotide interconversion,0.002879,NaN,0.031141
12,"Valine, leucine, and isoleucine metabolism",0.002096,NaN,0.031141
17,"Glycine, serine, alanine and threonine metabolism",0.003426,NaN,0.031141
27,Pentose phosphate pathway,0.008620,NaN,0.047020
45,Purine synthesis,0.005890,NaN,0.040161
59,"Transport, lysosomal",0.010499,NaN,0.047724


In [105]:
res_nvp_all[1][res_nvp_all[1]['q'] < 0.05]

,term,p,padj,q
5,Keratan sulfate synthesis,0.003612,NaN,0.016420
10,Inositol phosphate metabolism,0.000454,NaN,0.008258
14,Cholesterol metabolism,0.001923,NaN,0.014876
20,Aminosugar metabolism,0.008583,NaN,0.031209
43,Heme synthesis,0.002455,NaN,0.014876


In [103]:
res_nvp_all[1].to_csv(out_dir + 'results_Th17n_v_p_compass_rxns_q.csv')
res_pvn_all[1].to_csv(out_dir + 'results_Th17p_v_n_compass_rxns_q.csv')

In [96]:
for k,v in res_pvn_all.items():
    print(f"{k} genes: pvn: {len(v[v['q'] < 0.05])} nvp: {len(res_nvp_all[k][res_nvp_all[k]['q'] < 0.05])}")
    #display(v[v['q'] < 0.05])

1 genes: pvn: 6 nvp: 5
2 genes: pvn: 6 nvp: 5
3 genes: pvn: 6 nvp: 5
4 genes: pvn: 6 nvp: 5
5 genes: pvn: 3 nvp: 5
6 genes: pvn: 3 nvp: 5
7 genes: pvn: 3 nvp: 4
8 genes: pvn: 3 nvp: 4
9 genes: pvn: 3 nvp: 4
10 genes: pvn: 3 nvp: 4
11 genes: pvn: 3 nvp: 4
12 genes: pvn: 3 nvp: 4
13 genes: pvn: 3 nvp: 4
14 genes: pvn: 3 nvp: 4
15 genes: pvn: 3 nvp: 2


In [73]:
for k,v in res_pvn_all.items():
    print(f"{k}: {len(v[v['q'] < 0.05])}")
    display(v[v['q'] < 0.05])

1: 6


,term,p,padj,q
1,Nucleotide interconversion,0.002879,NaN,0.031141
12,"Valine, leucine, and isoleucine metabolism",0.002096,NaN,0.031141
17,"Glycine, serine, alanine and threonine metabolism",0.003426,NaN,0.031141
27,Pentose phosphate pathway,0.008620,NaN,0.047020
45,Purine synthesis,0.005890,NaN,0.040161
59,"Transport, lysosomal",0.010499,NaN,0.047724


2: 6


,term,p,padj,q
1,Nucleotide interconversion,0.002879,NaN,0.031141
12,"Valine, leucine, and isoleucine metabolism",0.002096,NaN,0.031141
17,"Glycine, serine, alanine and threonine metabolism",0.003426,NaN,0.031141
27,Pentose phosphate pathway,0.008620,NaN,0.047020
45,Purine synthesis,0.005890,NaN,0.040161
57,"Transport, lysosomal",0.010499,NaN,0.047724


3: 6


...

,term,p,padj,q
1,Nucleotide interconversion,0.002879,NaN,0.031141
10,"Valine, leucine, and isoleucine metabolism",0.002096,NaN,0.031141
13,"Glycine, serine, alanine and threonine metabolism",0.003426,NaN,0.031141


14: 3


,term,p,padj,q
1,Nucleotide interconversion,0.002879,0.035397,0.031141
10,"Valine, leucine, and isoleucine metabolism",0.002096,0.035397,0.031141
13,"Glycine, serine, alanine and threonine metabolism",0.003426,0.035397,0.031141


15: 3


,term,p,padj,q
1,Nucleotide interconversion,0.002879,0.033114,0.031141
9,"Valine, leucine, and isoleucine metabolism",0.002096,0.033114,0.031141
12,"Glycine, serine, alanine and threonine metabolism",0.003426,0.033114,0.031141


In [74]:
for k,v in res_nvp_all.items():
    print(f"{k}: {len(v[v['q'] < 0.05])}")
    display(v[v['q'] < 0.05])

1: 5


,term,p,padj,q
5,Keratan sulfate synthesis,0.003612,NaN,0.016420
10,Inositol phosphate metabolism,0.000454,NaN,0.008258
14,Cholesterol metabolism,0.001923,NaN,0.014876
20,Aminosugar metabolism,0.008583,NaN,0.031209
43,Heme synthesis,0.002455,NaN,0.014876


2: 5


,term,p,padj,q
5,Keratan sulfate synthesis,0.003612,NaN,0.016420
10,Inositol phosphate metabolism,0.000454,NaN,0.008258
14,Cholesterol metabolism,0.001923,NaN,0.014876
20,Aminosugar metabolism,0.008583,NaN,0.031209
43,Heme synthesis,0.002455,NaN,0.014876


3: 5


...

,term,p,padj,q
4,Keratan sulfate synthesis,0.003612,0.044554,0.021894
9,Inositol phosphate metabolism,0.000454,0.016805,0.008258
11,Cholesterol metabolism,0.001923,0.035579,0.017483
16,Aminosugar metabolism,0.008583,0.079389,0.039012


14: 4


,term,p,padj,q
4,Keratan sulfate synthesis,0.003612,0.037329,0.021894
9,Inositol phosphate metabolism,0.000454,0.014080,0.008258
11,Cholesterol metabolism,0.001923,0.029809,0.017483
16,Aminosugar metabolism,0.008583,0.066515,0.039012


15: 2


,term,p,padj,q
8,Inositol phosphate metabolism,0.000454,0.013171,0.008258
10,Cholesterol metabolism,0.001923,0.027886,0.017483


# reaction scoring

In [76]:
all_genes = list(compass_data.index)
overlapping_terms_rxn, n_removed_terms_rxn = get_overlapping_met_genes(reaction_sig, all_genes, verb = True)

13DAMPPOX	original: 3 | overlapping: 3 | removed: 0
2HBO	original: 5 | overlapping: 5 | removed: 0
2OXOADOXm	original: 4 | overlapping: 4 | removed: 0
34DHOXPEGOX	original: 6 | overlapping: 6 | removed: 0
34DHPHAMT	original: 1 | overlapping: 1 | removed: 0
34DHPLACOX	original: 4 | overlapping: 3 | removed: 1
34DHPLACOX_NADP_	original: 3 | overlapping: 2 | removed: 1
34DHXMANDACOX	original: 3 | overlapping: 3 | removed: 0
34DHXMANDACOX_NADP_	original: 2 | overlapping: 1 | removed: 1
34HPPOR	original: 1 | overlapping: 1 | removed: 0
3AIBTm	original: 1 | overlapping: 1 | removed: 0
3DSPHR	original: 1 | overlapping: 1 | removed: 0
3HAO	original: 1 | overlapping: 1 | removed: 0
3HBCDm	original: 1 | overlapping: 1 | removed: 0
3HKYNAKGAT	original: 2 | overlapping: 1 | removed: 1
3HLYTCL	original: 1 | overlapping: 1 | removed: 0
3HPCOAHYD	original: 1 | overlapping: 1 | removed: 0
3HXKYNOXDA	original: 2 | overlapping: 2 | removed: 0
3M4HDXPAC	original: 4 | overlapping: 3 | removed: 1
3MOX4HOXP

filter to at least 5 genes in signature & 70% of genes are retained

In [77]:
prop_removed_rxn = {}
for k,v in n_removed_terms_rxn.items():
    prop_removed_rxn[k] = (len(reaction_sig[k]) - v)/len(reaction_sig[k])
#plt.hist(prop_removed.values())

In [83]:
min_prop = 0.7
upper_range = 16
overlapping_terms_filter_all_rxn = {i:{} for i in range(1,upper_range)}
for min_length in range(1,upper_range):
    overlapping_terms_filter_rxn = {}
    for k,v in overlapping_terms_rxn.items():
        if (len(v) >= min_length) & (prop_removed_rxn[k] > min_prop):
            overlapping_terms_filter_rxn[k] = v
    overlapping_terms_filter_all_rxn[min_length] = overlapping_terms_filter_rxn

In [84]:
for k,v in overlapping_terms_filter_all_rxn.items():
    print(f'{k}: {len(v)}')

1: 1259
2: 549
3: 352
4: 230
5: 96
6: 71
7: 35
8: 21
9: 15
10: 10
11: 9
12: 9
13: 8
14: 7
15: 7


In [85]:
res_pvn_all_rxn = {}
res_nvp_all_rxn = {}
for k,v in overlapping_terms_filter_all_rxn.items():
    res_pvn_all_rxn[k], res_nvp_all_rxn[k] = format_hypergeom_res(get_hypergeometric_results_compass_data(de_genes_th_1, v))

In [97]:
for k,v in res_pvn_all_rxn.items():
    print(f"{k} genes: pvn: {len(v[v['q'] < 0.05])} nvp: {len(res_nvp_all_rxn[k][res_nvp_all_rxn[k]['q'] < 0.05])}")
    #display(v[v['q'] < 0.05])

1 genes: pvn: 430 nvp: 423
2 genes: pvn: 248 nvp: 333
3 genes: pvn: 212 nvp: 252
4 genes: pvn: 175 nvp: 164
5 genes: pvn: 0 nvp: 5
6 genes: pvn: 0 nvp: 5
7 genes: pvn: 0 nvp: 5
8 genes: pvn: 0 nvp: 5
9 genes: pvn: 0 nvp: 5
10 genes: pvn: 0 nvp: 0
11 genes: pvn: 0 nvp: 0
12 genes: pvn: 0 nvp: 0
13 genes: pvn: 0 nvp: 0
14 genes: pvn: 0 nvp: 0
15 genes: pvn: 0 nvp: 0


In [ ]:
sig_res_pvn = res_pvn_all_rxn[1]
sig_res_nvp = res_nvp_all_rxn[1]

In [99]:
res_nvp_all_rxn[1]

,term,p,padj,q
0,2HBO,0.469120,NaN,0.010935
1,2OXOADOXm,NaN,NaN,NaN
2,34DHPHAMT,NaN,NaN,NaN
3,3HBCDm,NaN,NaN,NaN
4,4HGLSDm,NaN,NaN,NaN
...,...,...,...,...
687,UGALGTg,0.316079,NaN,0.008281
688,URIK1,0.316079,NaN,0.008281
689,VACCCPT1,0.316079,NaN,0.008281
690,XYLTer,0.223741,NaN,0.007590


In [93]:
for k,v in res_pvn_all_rxn.items():
    print(f"{k}: {len(v[v['q'] < 0.05])}")
    display(v[v['q'] < 0.05])

1: 430


,term,p,padj,q
0,2HBO,0.108134,NaN,0.005105
1,2OXOADOXm,0.070256,NaN,0.005105
2,34DHPHAMT,0.117356,NaN,0.005105
3,3HBCDm,0.117356,NaN,0.005105
4,4HGLSDm,0.117356,NaN,0.005105
...,...,...,...,...
425,VALTAm,0.117356,NaN,0.005105
426,XYLUR,0.117356,NaN,0.005105
427,GNDc,0.117356,NaN,0.005105
428,P5CR,0.312380,NaN,0.009606


2: 248


,term,p,padj,q
0,2HBO,0.108134,NaN,0.018808
1,2OXOADOXm,0.070256,NaN,0.018808
2,5HOXINDACTOXm,0.312380,NaN,0.018808
3,ACACT10m,0.312380,NaN,0.018808
4,ADK1m,0.220945,NaN,0.018808
...,...,...,...,...
243,T2M26DCOAHLm,0.312380,NaN,0.018808
244,TRPO2,0.312380,NaN,0.018808
245,UGT1A2r,0.312380,NaN,0.018808
246,UGT1A5r2,0.393088,NaN,0.018808


3: 212


...

,term,p,padj,q


14: 0


,term,p,padj,q


15: 0


,term,p,padj,q


In [94]:
for k,v in res_nvp_all_rxn.items():
    print(f"{k}: {len(v[v['q'] < 0.05])}")
    display(v[v['q'] < 0.05])

1: 423


,term,p,padj,q
0,2HBO,0.469120,NaN,0.010935
5,5HOXINDACTOXm,0.316079,NaN,0.008281
7,ACACT10m,0.316079,NaN,0.008281
19,ADSS,0.223741,NaN,0.007590
20,AG13T10g,0.532279,NaN,0.011576
...,...,...,...,...
687,UGALGTg,0.316079,NaN,0.008281
688,URIK1,0.316079,NaN,0.008281
689,VACCCPT1,0.316079,NaN,0.008281
690,XYLTer,0.223741,NaN,0.007590


2: 333


,term,p,padj,q
0,2HBO,0.469120,NaN,0.014216
2,5HOXINDACTOXm,0.316079,NaN,0.011181
3,ACACT10m,0.316079,NaN,0.011181
5,ADSS,0.223741,NaN,0.011181
6,AG13T10g,0.532279,NaN,0.014753
...,...,...,...,...
415,TYRTA,0.223741,NaN,0.011181
416,UGALGTg,0.316079,NaN,0.011181
417,URIK1,0.316079,NaN,0.011181
418,VACCCPT1,0.316079,NaN,0.011181


3: 252


...

,term,p,padj,q


14: 0


,term,p,padj,q


15: 0


,term,p,padj,q
